# Sonar Visualization Playground

Quick-start notebook to run the 2D sonar simulator, view wavefields inline, and export GIFs for report/presentation assets.

In [ ]:
%matplotlib qt

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.embed_limit'] = 50  # MB, pick your size

from IPython.display import HTML, Image

from setup_sonar_model import setup_sonar_model, print_model_info
from simpleLeapFrog import LeapfrogSolver
from sonar_viz import plot_pressure_xz_at
from create_wave_animation import create_wave_animation


## Run a baseline simulation
Configure a moderate grid and stable timestep (fraction of CFL). Adjust `Nx/Nz` or `t_extra` for larger domains or longer runs.

In [ ]:
def run_simulation(
        Nx=201, Nz=51, Lz=500, f0=20, t_extra=-0.56,
        # t_final=0.8,
        dt_scale=0.5
        ):

    """Run sonar simulation and return state history"""
    # enforce square cells: set Lx from Lz and grid sizes
    Lx = Lz * (Nx - 1) / (Nz - 1)

    hydro_config = {
        "x_indices": [50, 75, 100, 125, 150],
        "z_indices": [15, 20, 25, 30, 35],
        "n_phones": 5
    }
    source_pos = (101, 26)

    model = setup_sonar_model(
        Nx=Nx, Nz=Nz, Lx=Lx, Lz=Lz, f0=f0, t_extra=t_extra,
        source_position=source_pos, hydrophone_config=hydro_config
    )

    # model['t_stop'] = t_final
    dt = dt_scale * model['max_dt_FE']
    num_steps = int(np.ceil((model['t_stop'] - model['t_start']) / dt))
    print_model_info(model, verbose=False)
    print(f"Using dt = {dt:.3e} s with {num_steps} steps")
    X, t = LeapfrogSolver(model['eval_f'], model['x_start'], model['p'], model['eval_u'], num_steps, dt, visualize=False, verbose=False)
    return {"model": model, "X": X, "t": t}

# Run a default scenario
sim = run_simulation()


## Single-frame snapshot
Inspect the pressure field at a chosen time (ms).

In [ ]:
from visualize_sonar import visualize_sonar_setup

# Domain layout snapshot for the current parameters
fig, ax = visualize_sonar_setup(sim['model']['p'], show_grid=True, show_boundary_labels=True)

## Fast GIF writer (imageio)
Bypass Matplotlib animation and write frames directly. Adjust `frame_skip`/`max_frames`/`fps` to trade speed vs. smoothness.

In [ ]:
import imageio.v2 as imageio
import matplotlib.cm as cm

def save_gif_fast(sim, gif_path, frame_skip=4, max_frames=300, fps=12, cmap_name="RdBu_r"):
    p = sim['model']['p']
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    pressure = sim['X'][:N, :].reshape(Nx, Nz, -1)
    v_abs = float(np.max(np.abs(pressure))) if pressure.size else 1.0
    cmap = cm.get_cmap(cmap_name)

    frames = []
    # Limit frames to keep runtime reasonable
    max_idx = min(pressure.shape[2], frame_skip * max_frames)
    for i in range(0, max_idx, frame_skip):
        frame = pressure[:, :, i].T
        norm = (frame + v_abs) / (2 * v_abs + 1e-12)  # map [-v_abs, v_abs] -> [0,1]
        rgba = (cmap(np.clip(norm, 0, 1)) * 255).astype(np.uint8)
        frames.append(rgba)

    imageio.mimsave(gif_path, frames, fps=fps)
    return gif_path

# Example fast export
os.makedirs('report/figs', exist_ok=True)
fast_gif_path = 'report/figs/sonar_wavefield_fast.gif'
# save_gif_fast(sim, fast_gif_path, frame_skip=4, max_frames=400, fps=12)
Image(filename=fast_gif_path)


## (Optional) Notes
- Increase `Nx/Nz` or `t_extra` for higher-fidelity visuals; adjust `frame_skip` to keep animations responsive.
- Swap in `getParam_Absorb.py` (sponge/PML) inside `setup_sonar_model` if you prefer fewer boundary reflections for visuals.
- For MOR comparisons, run a reduced-order model separately and feed both `X` arrays into a modified animation that overlays or plots residuals.

## Wavefield + Per-Hydrophone Lines
Wavefield on top with each hydrophone plotted in its own row underneath, all synced with a moving time cursor.

In [ ]:
%matplotlib qt

from eval_g_Sonar import eval_g_Sonar
import numpy as np

def make_wave_plus_per_channel(sim, frame_skip=2, interval=60, max_frames=300, stride=1):
    p, X, t = sim['model']['p'], sim['X'], sim['t']
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    dx, dz = p['dx'], p['dz']
    pressure = X[:N, :].reshape(Nx, Nz, -1)
    total_frames = pressure.shape[2]

    idx = list(range(0, total_frames, stride))
    if idx[-1] != total_frames - 1:
        idx.append(total_frames - 1)

    traces = []
    for i in idx:
        traces.append(eval_g_Sonar(X[:, i].reshape(-1, 1), p).reshape(-1))
    traces = np.stack(traces) if traces else np.zeros((0, 0))
    H = traces.shape[1] if traces.size else 0
    t_ms = t[idx] * 1000 if len(idx) else np.array([])

    if H == 0:
        print("No hydrophone data returned (n_phones=0 or indices out of bounds).")
        return HTML("<p>No hydrophone data.</p>")

    # Build figure with wavefield + one axis per hydrophone
    height_ratios = [3] + [1] * H
    fig, axes = plt.subplots(H + 1, 1, figsize=(8, 2 + 2 * H),
                             gridspec_kw={'height_ratios': height_ratios},
                             sharex=False)
    axes = np.atleast_1d(axes)
    ax_w = axes[0]

    v_abs = np.max(np.abs(pressure)) if pressure.size else 1.0
    im_w = ax_w.imshow(pressure[:, :, 0].T, extent=[0, p['Lx'], p['Lz'], 0],
                       cmap='RdBu_r', vmin=-v_abs, vmax=v_abs, aspect='equal')
    sx, sz = p['sonar_ix'] * dx, p['sonar_iz'] * dz
    ax_w.plot([sx], [sz], '*', color='yellow', markersize=10, markeredgecolor='k', label='Source')
    hp = p.get('hydrophones', {})
    if hp.get('n_phones', 0) > 0:
        if 'z_pos' in hp and 'x_indices' in hp:
            zpos = hp['z_pos'] * dz
            for i, xi in enumerate(hp['x_indices']):
                x_coord = xi * dx
                ax_w.plot(x_coord, zpos, '^', color='white', markersize=6, markeredgecolor='k', alpha=0.9)
                ax_w.text(x_coord + dx*0.5, zpos - dz*0.5, f'H{i+1}', color='white', fontsize=8, ha='left', va='center', weight='bold')
        elif 'x_pos' in hp and 'z_indices' in hp:
            xpos = hp['x_pos'] * dx
            for i, zi in enumerate(hp['z_indices']):
                z_coord = zi * dz
                ax_w.plot(xpos, z_coord, '^', color='white', markersize=6, markeredgecolor='k', alpha=0.9)
                ax_w.text(xpos + dx*0.5, z_coord - dz*0.5, f'H{i+1}', color='white', fontsize=8, ha='left', va='center', weight='bold')
        elif 'x_indices' in hp and 'z_indices' in hp:
            for i, (xi, zi) in enumerate(zip(hp['x_indices'], hp['z_indices'])):
                x_coord = xi * dx
                z_coord = zi * dz
                ax_w.plot(x_coord, z_coord, '^', color='white', markersize=6, markeredgecolor='k', alpha=0.9)
                # hydrophone index in wavefield plot
                ax_w.text(x_coord + dx*0.5, z_coord - dz*0.5, f'H{i+1}', color='black', fontsize=8, ha='left', va='center', weight='light')
    ax_w.set_xlabel('X (m)')
    ax_w.set_ylabel('Z (m)')

    colors = plt.cm.tab10(np.linspace(0, 1, max(H, 1)))
    cursors = []
    for k in range(H):
        ax = axes[k + 1]
        ax.plot(t_ms, traces[:, k], color=colors[k], label=f'H{k+1}')
        cur = ax.axvline(t_ms[0], color='red', linestyle='--', linewidth=1.2)
        cursors.append(cur)
        ax.set_ylabel('Pa')
        ax.grid(True, alpha=0.3)
        ax.legend(loc='upper right')
    axes[-1].set_xlabel('Time (ms)')
    plt.tight_layout()
    plt.close(fig)  # prevent duplicate static display

    def animate(i):
        frame_idx = min(i * frame_skip, len(idx) - 1)
        frame = idx[frame_idx]
        im_w.set_array(pressure[:, :, frame].T)
        ax_w.set_title(f't = {t_ms[frame_idx]:.1f} ms')
        for cur in cursors:
            cur.set_xdata([t_ms[frame_idx], t_ms[frame_idx]])
        return [im_w, *cursors]

    nframes = min(max_frames, (len(idx) + frame_skip - 1) // frame_skip)
    anim = animation.FuncAnimation(fig, animate, frames=nframes, interval=interval, blit=True)
    return HTML(anim.to_jshtml())

# Example
make_wave_plus_per_channel(sim, frame_skip=7, interval=60, max_frames=300, stride=1)


In [ ]:
%matplotlib qt

from eval_g_Sonar import eval_g_Sonar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

def make_wave_plus_per_channel_qt(sim, frame_skip=2, interval=60, max_frames=300, stride=1):
    p, X, t = sim['model']['p'], sim['X'], sim['t']
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    dx, dz = p['dx'], p['dz']
    Lx, Lz = p['Lx'], p['Lz']
    pressure = X[:N, :].reshape(Nx, Nz, -1)
    total_frames = pressure.shape[2]

    idx = list(range(0, total_frames, stride))
    if idx[-1] != total_frames - 1:
        idx.append(total_frames - 1)

    traces = []
    for i in idx:
        traces.append(eval_g_Sonar(X[:, i].reshape(-1, 1), p).reshape(-1))
    traces = np.stack(traces) if traces else np.zeros((0, 0))
    H = traces.shape[1] if traces.size else 0
    t_ms = t[idx] * 1000 if len(idx) else np.array([])

    if H == 0:
        print("No hydrophone data returned (n_phones=0 or indices out of bounds).")
        return

    # Calculate figure size based on domain aspect ratio
    domain_aspect = Lx / Lz
    plot_width = 40
    plot_height = plot_width / domain_aspect
    
    # Height ratios: wavefield gets more space
    height_ratios = [plot_height] + [1] * H
    total_height = plot_height + H + 2  # Extra for controls
    
    fig, axes = plt.subplots(H + 1, 1, figsize=(plot_width + 2, total_height),
                             gridspec_kw={'height_ratios': height_ratios},
                             sharex=False)
    axes = np.atleast_1d(axes)
    ax_w = axes[0]

    v_abs = np.max(np.abs(pressure)) if pressure.size else 1.0
    im_w = ax_w.imshow(pressure[:, :, 0].T, extent=[0, Lx, Lz, 0],
                       cmap='RdBu_r', vmin=-v_abs, vmax=v_abs, aspect='equal')
    
    # Colorbar
    cbar = plt.colorbar(im_w, ax=ax_w, orientation='vertical', label='Pressure (Pa)', 
                        fraction=0.02, pad=0.02)
    cbar.ax.tick_params(labelsize=8)
    
    sx, sz = p['sonar_ix'] * dx, p['sonar_iz'] * dz
    ax_w.plot([sx], [sz], '*', color='yellow', markersize=12, markeredgecolor='k', label='Source')
    
    hp = p.get('hydrophones', {})
    colors = plt.cm.tab10(np.linspace(0, 1, max(H, 1)))
    
    if hp.get('n_phones', 0) > 0:
        if 'z_pos' in hp and 'x_indices' in hp:
            zpos = hp['z_pos'] * dz
            for i, xi in enumerate(hp['x_indices']):
                x_coord = xi * dx
                ax_w.plot(x_coord, zpos, '^', color=colors[i], markersize=8, 
                         markeredgecolor='k', alpha=0.9, label=f'H{i+1}')
        elif 'x_pos' in hp and 'z_indices' in hp:
            xpos = hp['x_pos'] * dx
            for i, zi in enumerate(hp['z_indices']):
                z_coord = zi * dz
                ax_w.plot(xpos, z_coord, '^', color=colors[i], markersize=8,
                         markeredgecolor='k', alpha=0.9, label=f'H{i+1}')
        elif 'x_indices' in hp and 'z_indices' in hp:
            for i, (xi, zi) in enumerate(zip(hp['x_indices'], hp['z_indices'])):
                x_coord = xi * dx
                z_coord = zi * dz
                ax_w.plot(x_coord, z_coord, '^', color=colors[i], markersize=8,
                         markeredgecolor='k', alpha=0.9, label=f'H{i+1}')
    
    ax_w.set_xlabel('Range (m)')
    ax_w.set_ylabel('Depth (m)')
    title_w = ax_w.set_title(f't = {t_ms[0]:.1f} ms', fontsize=12)
    ax_w.legend(loc='upper right', fontsize=8, framealpha=0.9, ncol=2)

    cursors = []
    for k in range(H):
        ax = axes[k + 1]
        ax.plot(t_ms, traces[:, k], color=colors[k], lw=1.2, label=f'H{k+1}')
        cur = ax.axvline(t_ms[0], color='red', linestyle='--', linewidth=1.2)
        cursors.append(cur)
        ax.set_ylabel(f'H{k+1} (Pa)', fontsize=9)
        ax.grid(True, alpha=0.3)
        if k < H - 1:
            ax.set_xticklabels([])
    axes[-1].set_xlabel('Time (ms)')

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.10)

    # Add slider and play button
    ax_slider = plt.axes([0.2, 0.03, 0.5, 0.02])
    ax_button = plt.axes([0.75, 0.03, 0.1, 0.02])
    
    n_frames = min(max_frames, (len(idx) + frame_skip - 1) // frame_skip)
    slider = Slider(ax_slider, 'Frame', 0, n_frames - 1, valinit=0, valstep=1)
    button = Button(ax_button, 'Play')
    
    anim_running = [False]
    current_frame = [0]

    def update_frame(frame_num):
        frame_num = int(frame_num)
        frame_idx = min(frame_num * frame_skip, len(idx) - 1)
        frame = idx[frame_idx]
        
        im_w.set_array(pressure[:, :, frame].T)
        title_w.set_text(f't = {t_ms[frame_idx]:.1f} ms')
        
        for cur in cursors:
            cur.set_xdata([t_ms[frame_idx], t_ms[frame_idx]])
        
        fig.canvas.draw_idle()

    def on_slider_change(val):
        current_frame[0] = int(val)
        update_frame(val)

    def animate():
        if anim_running[0]:
            current_frame[0] = (current_frame[0] + 1) % n_frames
            slider.set_val(current_frame[0])
            timer.start(interval)

    def on_button_click(event):
        if anim_running[0]:
            anim_running[0] = False
            button.label.set_text('Play')
            timer.stop()
        else:
            anim_running[0] = True
            button.label.set_text('Pause')
            animate()

    slider.on_changed(on_slider_change)
    button.on_clicked(on_button_click)

    timer = fig.canvas.new_timer(interval=interval)
    timer.add_callback(animate)

    plt.show()

# Example usage
make_wave_plus_per_channel_qt(sim, frame_skip=7, interval=60, max_frames=300, stride=1)